## This notebook was made for a Kaggle competition

# https://www.kaggle.com/c/home-data-for-ml-course/overview

In [44]:
from IPython.display import Image
Image(url= "img/result.png")

In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

train = pd.read_csv('home-data/train.csv')
test = pd.read_csv('home-data/test.csv')

combine = [train,test]

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)


## Prepare Data

In [46]:
#Lets see how many missing data we have

print(combine[0].isnull().sum().sort_values(ascending = False))
print('---'*10)
print(combine[1].isnull().sum().sort_values(ascending = False))

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
LotFrontage       259
GarageYrBlt        81
GarageCond         81
GarageType         81
GarageFinish       81
GarageQual         81
BsmtFinType2       38
BsmtExposure       38
BsmtQual           37
BsmtCond           37
BsmtFinType1       37
MasVnrArea          8
MasVnrType          8
Electrical          1
Id                  0
Functional          0
Fireplaces          0
KitchenQual         0
KitchenAbvGr        0
BedroomAbvGr        0
HalfBath            0
FullBath            0
BsmtHalfBath        0
TotRmsAbvGrd        0
GarageCars          0
GrLivArea           0
GarageArea          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
BsmtFullBath        0
HeatingQC 

In [47]:
Image(url= "img/example.png")

In [48]:
# Replace all nan with a category NA based on the information we have 
for dataset in combine:
    dataset['PoolQC'] = dataset['PoolQC'].replace(np.nan, 'NA')
    dataset['Alley'] = dataset['Alley'].replace(np.nan, 'NA')
    dataset['Fence'] = dataset['Fence'].replace(np.nan, 'NA')
    dataset['MiscFeature'] = dataset['MiscFeature'].replace(np.nan, 'NA')
    dataset['FireplaceQu'] = dataset['FireplaceQu'].replace(np.nan, 'NA')
    dataset['GarageQual'] = dataset['GarageQual'].replace(np.nan, 'NA')
    dataset['GarageFinish'] = dataset['GarageFinish'].replace(np.nan, 'NA')
    dataset['GarageType'] = dataset['GarageType'].replace(np.nan, 'NA')
    dataset['GarageCond'] = dataset['GarageCond'].replace(np.nan, 'NA')
    dataset['BsmtCond'] = dataset['BsmtCond'].replace(np.nan, 'NA')
    dataset['BsmtExposure'] = dataset['BsmtExposure'].replace(np.nan, 'NA')
    dataset['BsmtFinType2'] = dataset['BsmtFinType2'].replace(np.nan, 'NA')
    dataset['BsmtFinType1'] = dataset['BsmtFinType1'].replace(np.nan, 'NA')
    dataset['BsmtQual'] = dataset['BsmtQual'].replace(np.nan, 'NA')
    dataset['MasVnrArea'] = dataset['MasVnrArea'].replace('None', 'NA')

In [49]:
X = combine[0].drop(['SalePrice'],axis=1)
y = combine[0]['SalePrice']
test = combine[1]

cat_cols = X.select_dtypes(exclude=['int64','int32','float64']).columns.tolist()
num_cols = X._get_numeric_data().columns
num_cols = num_cols.tolist()

In [50]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OneHotEncoder()),
])

full_pipeline = ColumnTransformer(
    transformers=[
        ('cat', cat_pipeline, cat_cols),
        ('num', num_pipeline, num_cols)
        ]
)


X_train = full_pipeline.fit_transform(X).toarray()
X_test = full_pipeline.transform(test).toarray()

## Training and prediction

In [51]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score

In [52]:
from sklearn import decomposition

pca = decomposition.PCA(n_components=200)
#X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)

In [53]:
lin_reg = LinearRegression()
lin_reg.fit(X_train,y)

scores = cross_val_score(lin_reg, X_train, y ,scoring="neg_mean_squared_error", cv=10)
lin_reg_scores = np.sqrt(-scores)
lin_reg_scores

array([8.93595473e+13, 1.81899218e+15, 1.48988538e+15, 2.16018173e+15,
       2.33269697e+14, 1.19521344e+15, 5.08869707e+14, 1.54725081e+15,
       4.33451843e+13, 1.50463668e+14])

In [55]:
SGD = SGDRegressor()
SGD.fit(X_train,y)

scores = cross_val_score(SGD, X_train, y ,scoring="neg_mean_squared_error", cv=10)
SGD_scores = np.sqrt(-scores)
SGD_scores

array([22240.29337049, 24876.71196477, 27853.41547993, 38955.59640356,
       34493.10788516, 26622.65467749, 25734.41737074, 24374.56164957,
       57544.68550947, 24463.23462005])

In [56]:
from sklearn.linear_model import ElasticNet

ENet = ElasticNet(random_state=0)
ENet.fit(X_train,y)

scores = cross_val_score(ENet, X_train, y ,scoring="neg_mean_squared_error", cv=10)
ENet_scores = np.sqrt(-scores)
ENet_scores



array([23254.16413109, 29516.6622036 , 24886.18869198, 39895.34787405,
       41092.05246643, 29890.3758231 , 26967.68098873, 26937.99278389,
       59537.19422265, 25072.91618646])

In [57]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X_train, y)

scores = cross_val_score(ridge_reg, X_train, y ,scoring="neg_mean_squared_error", cv=10)
ridge_reg_scores = np.sqrt(-scores)
ridge_reg_scores

array([20902.09994358, 26666.19204162, 25316.53509385, 39569.47580966,
       31353.05600329, 29686.57381733, 25639.1441609 , 24277.32010488,
       62636.66759066, 24960.13207897])

In [58]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_train, y)

scores = cross_val_score(lasso_reg, X_train, y ,scoring="neg_mean_squared_error", cv=10)
lasso_reg_scores = np.sqrt(-scores)
lasso_reg_scores

array([21927.68941547, 32366.76778302, 23761.43133623, 41245.04901463,
       30343.30593872, 43331.91832463, 24120.61218081, 22709.88650627,
       68158.66058159, 33194.38400647])

In [59]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

random_forest_reg = RandomForestRegressor(n_estimators=200, random_state=0)

para_grids = {
            "n_estimators" : [10,50,100],
            "max_features" : ["auto", "log2", "sqrt"],
            "bootstrap"    : [True, False]
        }


grid = GridSearchCV(random_forest_reg, para_grids)
grid.fit(X_train,y)
forest = grid.best_estimator_

scores = cross_val_score(forest, X_train, y ,scoring="neg_mean_squared_error", cv=5)
random_forest_reg_score = np.sqrt(-scores)
random_forest_reg_score

array([23840.2577669 , 31816.15414439, 33544.87081366, 23964.32896952,
       32936.93664371])

In [60]:
models = [ridge_reg_scores,lasso_reg_scores,ENet_scores,SGD_scores,lin_reg_scores,random_forest_reg_score]
tags = ['ridge_reg_score','lasso_reg_score','ENet_score','SGD_score','lin_reg_score','random_forest_reg_score']
for j in range(0,len(models)):
    print(tags[j] + ' : ' + str((sum(models[j])/len(models[j]))))

ridge_reg_score : 31100.719664472843
lasso_reg_score : 34115.970508785205
ENet_score : 32705.057537198292
SGD_score : 30715.86789312248
lin_reg_score : 923683135381992.0
random_forest_reg_score : 29220.509667635186


In [61]:
predictions = forest.predict(X_test)

output = pd.DataFrame()

output['Id'] ,output['SalePrice']= [j for j in range (1461,2920)] , predictions

output.to_csv('submit_housing', index=False)